In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from datasets import TripletAudio
from networks import AnchorNet, EmbeddingNet, TripletNet
from losses import TripletLoss
import torch.optim as optim
from recall import Recall
from torch.optim import lr_scheduler
from trainer import fit
from recall import Recall
import itertools
from tensorboardX import SummaryWriter

In [2]:
#define hyperparams
K, MAX_CLOSE_NEG, P_STRONG_NEG = 5, 15, 1
BATCH_SIZE = 128
INPUT_D, OUTPUT_D = 192, 128
MARGIN, N_EPOCHS, LOG_INT, N_RECALL_CAND = 0.5, 20, 100, 25
N_RECALL_SAMPLES = 1000
LEARNING_RATE, STEP_SIZE, GAMMA = 1e-3, 15, 0.1

In [3]:
#computes the recall on datasets using global variables such as model and recall
def measure_recall(datasets):
    outputs = []
    for d in datasets:
        batch_indicies = np.random.choice(list(range(0, d.get_dataset().shape[0])), N_RECALL_SAMPLES, False)
        queries = d.get_dataset()[batch_indicies]
        knns_of_batch = d.get_knn().iloc[batch_indicies, :]
        outputs.append(recall.calculate(d.get_dataset(), model.embedding_net, queries, knns_of_batch, False))
    return [np.mean(x) for x in outputs]

In [ ]:
# for (LEARNING_RATE, STEP_SIZE, GAMMA) in itertools.product(*hyperparams):
#setup datasets
cuda = torch.cuda.is_available()
triplet_train_dataset = TripletAudio(True, K, MAX_CLOSE_NEG, P_STRONG_NEG)
triplet_test_dataset = TripletAudio(False, K, MAX_CLOSE_NEG, P_STRONG_NEG)
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)
#define model 
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)
if cuda:
    print("GPU Enabled")
    model.cuda()
loss_fn = TripletLoss(MARGIN)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA, last_epoch=-1)
recall = Recall(N_RECALL_CAND, K)
#run the model
train_loss, val_loss = fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, cuda, LOG_INT)
#measure recall
train_recall, val_recall = measure_recall([triplet_train_dataset, triplet_test_dataset])
#write to tensorboard
with SummaryWriter() as w:
    w.add_hparams(
        {'LEARNING_RATE': LEARNING_RATE, 'GAMMA': GAMMA, 'STEP_SIZE': STEP_SIZE, 'BSIZE': BATCH_SIZE, 'N_RECALL_S': N_RECALL_SAMPLES, 'N_RECALL_CAND': N_RECALL_CAND,
             'CLOSE_NEG': MAX_CLOSE_NEG, 'P_STRONG_NEG': P_STRONG_NEG, 'OUTPUT_D': OUTPUT_D},
        {'TRAIN_L': train_loss, 'VAL_L': val_loss, 'TRAIN_RECALL': train_recall, 'VAL_RECALL': val_recall})

In [ ]:
import matplotlib
matplotlib.pyplot.pie([len(x) for x in recall.bucket_hash.hash.values()]);

In [ ]:
temp = torch.randn(50,192)
output = embedding_net(temp)
output.shape